In [78]:
import json
import json5
import demjson

In [112]:
with open('../../data/australian_user_reviews.json', 'r') as f:
    # user_reviews = demjson.decode(f.read())
    user_reviews = json.load(f)


In [113]:
user_reviews[0]

{'user_id': '76561197970982479',
 'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
 'reviews': [{'funny': '',
   'posted': 'Posted November 5, 2011.',
   'last_edited': '',
   'item_id': '1250',
   'helpful': 'No ratings yet',
   'recommend': True},
  {'funny': '',
   'posted': 'Posted July 15, 2011.',
   'last_edited': '',
   'item_id': '22200',
   'helpful': 'No ratings yet',
   'recommend': True},
  {'funny': '',
   'posted': 'Posted April 21, 2011.',
   'last_edited': '',
   'item_id': '43110',
   'helpful': 'No ratings yet',
   'recommend': True}]}

In [114]:

with open('../../data/australian_users_items.json', 'r') as f:
    # user_items = demjson.decode(f.read())
    user_items = json.load(f)


In [115]:
user_items[0]

{'user_id': 'Wackky',
 'items_count': 0,
 'steam_id': '76561198039117046',
 'user_url': 'http://steamcommunity.com/id/Wackky',
 'items': []}

In [134]:
with open('../../data/steam_games.json', 'r') as f:
    # steam_games = demjson.decode(f.read())
    steam_games = json.load(f)

# with open('../../data/steam_review.json', 'r') as f:
#     steam_reviews = demjson.decode(f.read())

In [ ]:
steam_games[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [85]:
# {'publisher': 'Kotoshiro',
#  'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
#  'app_name': 'Lost Summoner Kitty',
#  'title': 'Lost Summoner Kitty',
#  'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
#  'release_date': '2018-01-04',
#  'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
#  'discount_price': 4.49,
#  'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
#  'specs': ['Single-player'],
#  'price': 4.99,
#  'early_access': False,
#  'id': '761140',
#  'developer': 'Kotoshiro'}
        

game_details = {
    game.get('id', None): {
        'developer': game.get('developer', None),
        'tags': game.get('tags', None)
    } for game in steam_games if game.get('id') is not None 
}



user_data = {}
user_game_reviews = {}
user_data_map = {}

for user in user_reviews:
    user_id = user['user_id']
    user_data[user_id] = {
        # 'user_url': user['user_url'],
        'reviews': []
    }

    for review in user['reviews']:
        game_id = review['item_id']
        game_info = game_details.get(game_id, {})
        user_data[user_id]['reviews'].append({
            'game_id': game_id,
            'posted': review['posted'],
            'recommend': review['recommend']
        })

for user in user_reviews:
    user_id = user['user_id']
    user_data_map[user_id] = {
        'reviews': {}
    }

    for review in user['reviews']:
        game_id = review['item_id']
        game_info = game_details.get(game_id, {})
        user_data_map[user_id][game_id] = {
            'posted': review['posted'],
            'recommend': review['recommend']
        }



for user in user_items:
    user_id = user['user_id']
    if user_id in user_data: 
        user_data[user_id]['items'] = []
        for item in user['items']:
            game_id = item['item_id']
            game_info = game_details.get(game_id, {})
            user_data[user_id]['items'].append({
                'game_id': game_id,
                'playtime_forever': item['playtime_forever'],
            })


In [86]:
# output to json
with open('../../data/user_data.json', 'w') as f:
    f.write(json.dumps(user_data, indent=4))

with open('../../data/user_data_map.json', 'w') as f:
    f.write(json.dumps(user_data_map, indent=4))

# game_details
with open('../../data/game_details.json', 'w') as f:
    f.write(json.dumps(game_details, indent=4))

# user_game_reviews
with open('../../data/user_game_reviews.json', 'w') as f:
    f.write(json.dumps(user_game_reviews, indent=4))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

data = []
for user_id, user_info in user_data.items():
    if 'items' not in user_info:
        continue
    
    for item in user_info['items']:
        game_id = item['game_id']
        playtime = item['playtime_forever']
        review = next((r for r in user_info['reviews'] if r['game_id'] == game_id), None)
        if review:
            recommend = review['recommend']
            
            game_reviews = user_game_reviews.get(game_id, [])
            overall_positive_rate = sum(1 for r in game_reviews if r['recommend']) / len(game_reviews) if game_reviews else 0
            
            user_positive_rate = sum(1 for r in user_info['reviews'] if r['recommend']) / len(user_info['reviews'])
            
            # 游戏分类的Jaccard相似性
            user_tags = set(tag for r in user_info['reviews'] for tag in game_details.get(r['game_id'], {}).get('tags', []))
            game_tags = set(game_details.get(game_id, {}).get('tags', []))
            tag_jaccard = len(user_tags & game_tags) / len(user_tags | game_tags) if user_tags | game_tags else 0
            
            # 游戏开发者的Jaccard相似性
            user_devs = set(game_details.get(r['game_id'], {}).get('developer', None) for r in user_info['reviews'])
            game_dev = game_details.get(game_id, {}).get('developer', None)
            dev_jaccard = int(game_dev in user_devs)  # 1 if matches, 0 otherwise
            
            data.append({
                'user_id': user_id,
                'game_id': game_id,
                'playtime_forever': playtime,
                'overall_positive_rate': overall_positive_rate,
                'user_positive_rate': user_positive_rate,
                'tag_jaccard': tag_jaccard,
                'dev_jaccard': dev_jaccard,
                'recommend': int(recommend)
            })

df = pd.DataFrame(data)

X = df[['playtime_forever', 'overall_positive_rate', 'user_positive_rate', 'tag_jaccard', 'dev_jaccard']]
y = df['recommend']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.78      0.88      0.82         8

    accuracy                           0.70        10
   macro avg       0.39      0.44      0.41        10
weighted avg       0.62      0.70      0.66        10



In [ ]:
def predict_recommendation(user_id, game_id):
    user_info = user_data.get(user_id, {})
    game_info = game_details.get(game_id, {})
    
    playtime = next((item['playtime_forever'] for item in user_info.get('items', []) if item['game_id'] == game_id), 0)
    
    overall_positive_rate = sum(1 for r in user_game_reviews.get(game_id, []) if r['recommend']) / len(user_game_reviews.get(game_id, [])) if user_game_reviews.get(game_id, []) else 0
    user_positive_rate = sum(1 for r in user_info.get('reviews', []) if r['recommend']) / len(user_info.get('reviews', [])) if user_info.get('reviews', []) else 0
    
    user_tags = set(tag for r in user_info.get('reviews', []) for tag in game_details.get(r['game_id'], {}).get('tags', []))
    game_tags = set(game_info.get('tags', []))
    tag_jaccard = len(user_tags & game_tags) / len(user_tags | game_tags) if user_tags | game_tags else 0
    
    user_devs = set(game_details.get(r['game_id'], {}).get('developer', None) for r in user_info.get('reviews', []))
    game_dev = game_info.get('developer', None)
    dev_jaccard = int(game_dev in user_devs)
    
    features = [[playtime, overall_positive_rate, user_positive_rate, tag_jaccard, dev_jaccard]]
    return model.predict(features)[0]


1


/home/commcheck/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [91]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

model1 = LogisticRegression()
model2 = RandomForestClassifier()
model3 = SVC()

voting_clf = VotingClassifier(estimators=[('lr', model1), ('rf', model2), ('svc', model3)], voting='hard')

voting_clf.fit(X_train, y_train)

y_pred_voting = voting_clf.predict(X_test)
print(classification_report(y_test, y_pred_voting))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.80      1.00      0.89         8

    accuracy                           0.80        10
   macro avg       0.40      0.50      0.44        10
weighted avg       0.64      0.80      0.71        10



/home/commcheck/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/commcheck/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/commcheck/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and be